In [3]:
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import plotly.express as px
import pydicom as dicom
import pydicom
import json
import glob
import collections
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
#from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import plotly.graph_objects as go
import random
from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [4]:
path = 'spine/'
label_coordinates_df = pd.read_csv(path + 'train_label_coordinates.csv')
train_series = pd.read_csv(path + 'train_series_descriptions.csv')
df_train = pd.read_csv(path + 'train.csv')
df_sub = pd.read_csv(path + 'sample_submission.csv')
test_series = pd.read_csv(path + 'test_series_descriptions.csv')

In [5]:
df_sub.head()

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.333333,0.333333,0.333333
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.333333,0.333333,0.333333
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.333333,0.333333,0.333333
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.333333,0.333333,0.333333
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.333333,0.333333,0.333333


In [6]:
df_sub.shape,test_series.shape,train_series.shape,label_coordinates_df.shape,df_train.shape

((25, 4), (3, 3), (6294, 3), (48692, 7), (1975, 26))

In [7]:
test_series.head()

,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [8]:
train_series.head()

,study_id,series_id,series_description
0,4003253,702807833,Sagittal T2/STIR
1,4003253,1054713880,Sagittal T1
2,4003253,2448190387,Axial T2
3,4646740,3201256954,Axial T2
4,4646740,3486248476,Sagittal T1


In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier


In [20]:
df_train_melted = df_train.melt(id_vars=['study_id'], var_name='condition_level', value_name='severity')
df_train_melted

,study_id,condition_level,severity
0,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild
1,4646740,spinal_canal_stenosis_l1_l2,Normal/Mild
2,7143189,spinal_canal_stenosis_l1_l2,Normal/Mild
3,8785691,spinal_canal_stenosis_l1_l2,Normal/Mild
4,10728036,spinal_canal_stenosis_l1_l2,Normal/Mild
...,...,...,...
49370,4282019580,right_subarticular_stenosis_l5_s1,Moderate
49371,4283570761,right_subarticular_stenosis_l5_s1,Normal/Mild
49372,4284048608,right_subarticular_stenosis_l5_s1,Normal/Mild
49373,4287160193,right_subarticular_stenosis_l5_s1,Normal/Mild


In [21]:
df_train_melted['level']=df_train_melted['condition_level'].apply(lambda x: '_'.join(x.split('_')[-2:]))

In [22]:
df_train_melted['condition']=df_train_melted['condition_level'].str.rsplit('_', n=2, expand=True).iloc[:,0]
df_train_melted

,study_id,condition_level,severity,level,condition
0,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis
1,4646740,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis
2,7143189,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis
3,8785691,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis
4,10728036,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis
...,...,...,...,...,...
49370,4282019580,right_subarticular_stenosis_l5_s1,Moderate,l5_s1,right_subarticular_stenosis
49371,4283570761,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis
49372,4284048608,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis
49373,4287160193,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis


In [23]:
le_severity = LabelEncoder()

In [24]:
df_train_melted['severity_encoded'] = le_severity.fit_transform(df_train_melted['severity'])
df_train_melted

,study_id,condition_level,severity,level,condition,severity_encoded
0,4003253,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
1,4646740,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
2,7143189,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
3,8785691,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
4,10728036,spinal_canal_stenosis_l1_l2,Normal/Mild,l1_l2,spinal_canal_stenosis,1
...,...,...,...,...,...,...
49370,4282019580,right_subarticular_stenosis_l5_s1,Moderate,l5_s1,right_subarticular_stenosis,0
49371,4283570761,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis,1
49372,4284048608,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis,1
49373,4287160193,right_subarticular_stenosis_l5_s1,Normal/Mild,l5_s1,right_subarticular_stenosis,1


In [25]:
X_train = df_train_melted[['study_id', 'condition', 'level']]
y_train = df_train_melted['severity_encoded']
X_train = pd.get_dummies(X_train, columns=['condition', 'level'])
X_train

,study_id,condition_left_neural_foraminal_narrowing,condition_left_subarticular_stenosis,condition_right_neural_foraminal_narrowing,condition_right_subarticular_stenosis,condition_spinal_canal_stenosis,level_l1_l2,level_l2_l3,level_l3_l4,level_l4_l5,level_l5_s1
0,4003253,False,False,False,False,True,True,False,False,False,False
1,4646740,False,False,False,False,True,True,False,False,False,False
2,7143189,False,False,False,False,True,True,False,False,False,False
3,8785691,False,False,False,False,True,True,False,False,False,False
4,10728036,False,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
49370,4282019580,False,False,False,True,False,False,False,False,False,True
49371,4283570761,False,False,False,True,False,False,False,False,False,True
49372,4284048608,False,False,False,True,False,False,False,False,False,True
49373,4287160193,False,False,False,True,False,False,False,False,False,True


In [26]:
y_train

0        1
1        1
2        1
3        1
4        1
        ..
49370    0
49371    1
49372    1
49373    1
49374    1
Name: severity_encoded, Length: 49375, dtype: int32

In [27]:
test_rows = []
for _, row in test_series.iterrows():
    for condition in ['left_neural_foraminal_narrowing', 'right_neural_foraminal_narrowing', 'left_subarticular_stenosis', 'right_subarticular_stenosis', 'spinal_canal_stenosis']:
        for level in ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']:
            test_rows.append({
                'study_id': row['study_id'],
                'condition': condition,
                'level': level
            })

X_test = pd.DataFrame(test_rows)
X_test = pd.get_dummies(X_test, columns=['condition', 'level'])
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
X_test

,study_id,condition_left_neural_foraminal_narrowing,condition_left_subarticular_stenosis,condition_right_neural_foraminal_narrowing,condition_right_subarticular_stenosis,condition_spinal_canal_stenosis,level_l1_l2,level_l2_l3,level_l3_l4,level_l4_l5,level_l5_s1
0,44036939,True,False,False,False,False,True,False,False,False,False
1,44036939,True,False,False,False,False,False,True,False,False,False
2,44036939,True,False,False,False,False,False,False,True,False,False
3,44036939,True,False,False,False,False,False,False,False,True,False
4,44036939,True,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
70,44036939,False,False,False,False,True,True,False,False,False,False
71,44036939,False,False,False,False,True,False,True,False,False,False
72,44036939,False,False,False,False,True,False,False,True,False,False
73,44036939,False,False,False,False,True,False,False,False,True,False


In [28]:
X_train

,study_id,condition_left_neural_foraminal_narrowing,condition_left_subarticular_stenosis,condition_right_neural_foraminal_narrowing,condition_right_subarticular_stenosis,condition_spinal_canal_stenosis,level_l1_l2,level_l2_l3,level_l3_l4,level_l4_l5,level_l5_s1
0,4003253,False,False,False,False,True,True,False,False,False,False
1,4646740,False,False,False,False,True,True,False,False,False,False
2,7143189,False,False,False,False,True,True,False,False,False,False
3,8785691,False,False,False,False,True,True,False,False,False,False
4,10728036,False,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
49370,4282019580,False,False,False,True,False,False,False,False,False,True
49371,4283570761,False,False,False,True,False,False,False,False,False,True
49372,4284048608,False,False,False,True,False,False,False,False,False,True
49373,4287160193,False,False,False,True,False,False,False,False,False,True


In [29]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [32]:
predictions_proba = model.predict_proba(X_test)
predictions_df = pd.DataFrame(predictions_proba, columns=le_severity.classes_)
predictions_df['study_id'] = X_test['study_id'].values
predictions_df['condition_level'] = X_test.index.map(lambda idx: f"{test_rows[idx]['condition']}_{test_rows[idx]['level']}")
predictions_df['row_id'] = predictions_df['study_id'].astype(str) + '_' + predictions_df['condition_level']
predictions_df

,Moderate,Normal/Mild,Severe,NaN,study_id,condition_level,row_id
0,0.00,1.00,0.00,0.0,44036939,left_neural_foraminal_narrowing_l1_l2,44036939_left_neural_foraminal_narrowing_l1_l2
1,0.00,1.00,0.00,0.0,44036939,left_neural_foraminal_narrowing_l2_l3,44036939_left_neural_foraminal_narrowing_l2_l3
2,0.09,0.91,0.00,0.0,44036939,left_neural_foraminal_narrowing_l3_l4,44036939_left_neural_foraminal_narrowing_l3_l4
3,0.09,0.91,0.00,0.0,44036939,left_neural_foraminal_narrowing_l4_l5,44036939_left_neural_foraminal_narrowing_l4_l5
4,0.04,0.96,0.00,0.0,44036939,left_neural_foraminal_narrowing_l5_s1,44036939_left_neural_foraminal_narrowing_l5_s1
...,...,...,...,...,...,...,...
70,0.00,1.00,0.00,0.0,44036939,spinal_canal_stenosis_l1_l2,44036939_spinal_canal_stenosis_l1_l2
71,0.09,0.90,0.01,0.0,44036939,spinal_canal_stenosis_l2_l3,44036939_spinal_canal_stenosis_l2_l3
72,0.29,0.71,0.00,0.0,44036939,spinal_canal_stenosis_l3_l4,44036939_spinal_canal_stenosis_l3_l4
73,0.02,0.87,0.11,0.0,44036939,spinal_canal_stenosis_l4_l5,44036939_spinal_canal_stenosis_l4_l5


In [34]:
normal_mild_value = (predictions_df['Normal/Mild'])
moderate_value = (predictions_df['Moderate'])
severe_value = (predictions_df['Severe'])

df_sub['normal_mild'] = normal_mild_value/2.3
df_sub['moderate'] =moderate_value*1.47
df_sub['severe'] =  (1-(normal_mild_value/2.3 +moderate_value*1.47))+severe_value-severe_value

In [35]:
df_sub

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.434783,0.0000,0.565217
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.434783,0.0000,0.565217
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.395652,0.1323,0.472048
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.395652,0.1323,0.472048
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.417391,0.0588,0.523809
5,44036939_left_subarticular_stenosis_l1_l2,0.434783,0.0000,0.565217
6,44036939_left_subarticular_stenosis_l2_l3,0.434783,0.0000,0.565217
7,44036939_left_subarticular_stenosis_l3_l4,0.152174,0.9555,-0.107674
8,44036939_left_subarticular_stenosis_l4_l5,0.147826,0.9702,-0.118026
9,44036939_left_subarticular_stenosis_l5_s1,0.260870,0.5880,0.151130


In [36]:
df_sub.to_csv('submission.csv', index=False)